In [6]:
import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings('ignore')
import requests

# Obtendo e Tratando Dados

In [7]:
url = 'https://www.fundamentus.com.br/resultado.php'

In [8]:
header = {
  "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"  
 }

r = requests.get(url, headers=header)

In [9]:
df = pd.read_html(r.text,  decimal=',', thousands='.')[0]

In [14]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
26,OMGE3,39.47,-3478.47,2.08,7.438,0.0000,0.873,4.57,24.12,-2.59,33.05,18.81,0.3084,0.0045,5.14,0.0457,-0.0006,43234400.0,3.722190e+09,1.24,0.5899
27,LINX3,37.50,-1491.57,4.22,8.214,0.0030,2.634,10.34,51.44,32.36,49.68,23.62,0.1597,-0.0055,2.26,0.0725,-0.0028,104302000.0,1.684040e+09,0.31,0.1583
33,STBP3,5.22,-254.55,2.16,5.007,0.0026,1.065,4.59,276.73,-5.14,236.47,23.43,0.0181,-0.0197,4.39,0.0053,-0.0085,28017500.0,2.084210e+09,0.21,0.0309
34,BTOW3,77.04,-246.06,4.52,4.729,0.0000,2.261,5.83,142.48,15.23,132.74,45.38,0.0332,-0.0192,2.49,0.0392,-0.0184,416663000.0,9.535270e+09,0.53,-0.0080
40,VVAR3,16.59,-142.47,4.71,0.980,0.0000,0.882,11.35,12.43,-4.01,15.25,10.47,0.0788,-0.0069,1.15,0.0993,-0.0330,914026000.0,5.632000e+09,1.44,0.0801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
917,SQIA3,22.59,772.89,3.68,8.007,0.0000,2.607,5.28,-423.29,8.32,-354.47,73.95,-0.0189,0.0106,5.49,-0.0140,0.0048,13049800.0,4.335560e+08,0.19,0.2390
921,APER3,31.20,3511.53,2.00,3.555,0.0017,1.216,9.88,-1072.38,-9.70,-915.96,47.58,-0.0033,0.0010,1.86,-0.0014,0.0006,1027750.0,1.790790e+08,0.00,-0.0808
922,BIDI3,33.90,19543.50,7.97,0.000,0.0015,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0004,9646980.0,3.251960e+09,0.00,0.1723
923,BIDI11,102.20,19639.60,8.01,0.000,0.0015,0.000,0.00,0.00,0.00,0.00,0.00,0.0000,0.0000,0.00,0.0000,0.0004,79453000.0,3.251960e+09,0.00,0.1723


In [11]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')
  df[coluna] = df[coluna].str.replace(',', '.')
  df[coluna] = df[coluna].str.rstrip('%').astype('float')/100

# Analisando Dados

In [13]:
df = df[df['Liq.2meses'] > 1000000]

In [16]:
ranking = pd.DataFrame()
ranking['pos'] = range(1, 151)
ranking['EV/EBIT'] = df[df['EV/EBIT'] > 0].sort_values(by=['EV/EBIT'])['Papel'][:150].values
ranking['ROIC'] = df.sort_values(by=['ROIC'], ascending=False)['Papel'][:150].values

In [17]:
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,GPIV33,PSSA3
2,3,SULA11,ODPV3
3,4,WIZS3,QUAL3
4,5,MRFG3,MRFG3
...,...,...,...
145,146,WEGE3,PTBL3
146,147,BOAS3,LIGT3
147,148,POSI3,BRPR3
148,149,COGN3,PRIO3


In [18]:
a = ranking.pivot_table(columns='EV/EBIT', values='pos')
b = ranking.pivot_table(columns='ROIC', values='pos')

In [19]:
t = pd.concat([a, b])
t

,ABEV3,AGRO3,ALPA4,ALSO3,ALUP11,AMAR3,ANIM3,B3SA3,BEEF3,BOAS3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,CAML3,CARD3,CCPR3,CCRO3,CESP6,CGAS5,CIEL3,CMIG3,CMIG4,COGN3,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CYRE3,DIRR3,DTEX3,ECOR3,EGIE3,ELET3,ELET6,ENAT3,...,SHUL4,SLCE3,SMLS3,SMTO3,SULA11,SUZB3,TAEE11,TAEE4,TASA3,TASA4,TEND3,TESA3,TGMA3,TIET11,TIET4,TIMS3,TOTS3,TRIS3,TRPL4,TUPY3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VIVA3,VIVT3,VIVT4,VLID3,VULC3,VVAR3,WEGE3,WIZS3,WSON33,YDUQ3,BTOW3,LWSA3,MGLU3,PRIO3,RDOR3
pos,88,54,144,82,17,132,124,116,9,147,111,57,72,97,122,55,46,60,86,84,73,76,23,19,149.0,43,14,18,48,106,20,58,107,36,130,53,59,25,26,7,...,83,45,31,75,3,114,42,41,30,29,32,15,117,50,51,49,81,74,21,79,115,38,95,91,16,139,71,69,78,120,80,146,4,24,109,NaN,NaN,NaN,NaN,NaN
pos,21,71,35,140,44,143,101,16,10,20,116,77,122,144,148,53,19,125,85,126,9,121,56,55,NaN,41,59,60,45,112,46,32,120,123,118,39,27,96,95,47,...,90,58,57,70,33,108,30,31,8,7,40,63,69,15,14,72,6,62,28,114,107,26,129,130,12,37,110,111,138,115,68,11,1,25,93,142.0,103.0,117.0,149.0,119.0


In [20]:
rank = t.dropna(axis=1).sum()

In [22]:
rank.sort_values()[:15]

PSSA3      3
WIZS3      5
MRFG3     10
BEEF3     19
JBSS3     28
VALE3     28
EQTL3     30
SAPR4     33
SAPR11    36
SULA11    36
TASA4     36
TASA3     38
JSLG3     40
SAPR3     45
TRPL4     49
dtype: int64